Autoren: Josephine Leib, Sabrina Lange

# Packageimport

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.lines as mlines
from matplotlib import pylab

# macht die Plots größer
plt.rcParams['figure.figsize'] = (16,8)

ModuleNotFoundError: No module named 'pandas'

# Daten einlesen und erkunden

In [ ]:
# als Index wird die Spalte "dateRep" verwendet, die mithilfe von parse_dates in einen Date-Time-Format umgewandelt wurde
# der zusätzliche Parameter dayfirst sorgt dafür, dass der Index im Format YYYY-MM-DD angezeigt wird
covid19 = pd.read_csv('Daten.csv', index_col='dateRep', parse_dates=['dateRep'], dayfirst = True)

# mithilfe von sort_index() wird sichergestellt, dass der "älteste" Eintrag "oben" und der jüngste Eintrag am Ende des Dataframes steht
covid19 = covid19.sort_index()

## Übersicht über die ersten Einträge des Datensatzes

In [ ]:
covid19.head()

- da die Indexspalte bereits alle notwendigen Datumsinformationen enthält, werden die Spalten "day", "month" und "year nicht mehr benötigt und gelöscht

In [ ]:
del covid19['day']
del covid19['month']
del covid19['year']

- der veränderte Datensatz hat nun folgende Gestalt

In [ ]:
covid19.head()

## Informationen zum Datensatz

- Größe des Datensatzes
- Spaltenbezeichnungen
- Datentypen
- statistische Kennzahlen

In [ ]:
print('Der Datensatz hat folgende Größe:\n' + 'Anzahl Instanzen: ' + str(covid19.shape[0]) + '\n' + 'Anzahl Spalten:   ' + str(covid19.shape[1]))

In [ ]:
covid19.info()

**Auffälligkeiten:**
- fehlende Werte in 3 Spalten

In [ ]:
covid19.describe()

**Auffälligkeiten:**
- Minimum in den Spalten 'cases' und 'deaths' ist negativ

## Überprüfen auf fehlende Werte

In [ ]:
missing_values_gesamt = covid19.isnull().sum().to_frame(name = 'Anzahl fehlender Werte')
missing_values = missing_values_gesamt[missing_values_gesamt['Anzahl fehlender Werte'] > 0]
# Anzeigen der Spalten, bei denen mindestens ein Wert fehlt
missing_values

### Betrachtung der Spalte 'geoId'

In [ ]:
covid19[covid19.geoId.isnull()].head()

- Vermutung: fehlende Werte betreffen ausschließlich Namibia


In [ ]:
# Probe
covid19[covid19.countryterritoryCode =='NAM'].shape
# --> es gibt 62 Einträge für Namibia

In [ ]:
# Probe
covid19[np.logical_and(covid19.geoId.isnull(), covid19.countryterritoryCode =='NAM')].shape
# --> bei allen 62 fehlenden Werten für 'geoID' ist auch 'countryterritoryCode' 'NAM' gesetzt

**Lösung**
- für Namibia wird manuell eine neue geoId 'NB' vergeben
- dafür wurde mithilfe der Funktion .unique() überprüft, welche 'geoId' noch nicht vergeben wurde

In [ ]:
np.sort(list(covid19['geoId'].unique()))

In [ ]:
# ersetze die fehlenden Werte in Spalte 'geoId' mit 'NB' und vollziehe die Änderung unmittelbar im Dataframe covid19
covid19['geoId'].fillna('NB', inplace = True)

### Betrachtung der Spalte 'countryterritoryCode'

In [ ]:
# Welche Instanzen weisen fehlende Werte auf?
mis_val_territory = covid19[covid19.countryterritoryCode.isnull()]
mis_val_territory

In [ ]:
# Wie viele Einträge je 'countriesAndTerritories' fehlen?
mis_val_territory['countriesAndTerritories'].value_counts()

- für diese 5 Fälle werden manuell neue 'countryterritoryCode' vergeben, die sich nicht mit den bisherigen überschneiden
- die Überprüfung hierfür erfolgt mithilfe der Funktion .unique()

In [ ]:
np.sort(list(covid19['countryterritoryCode'].unique()))

**Lösung**

Folgende Codes werden vergeben

- ICJ: Cases_on_an_international_conveyance_Japan
- ANG: Anguilla
- BON: Bonaire, Saint Eustatius and Saba
- FAL: Falkland_Islands_(Malvinas)
- WSH: Western_Sahara

In [ ]:
covid19.loc[covid19['countriesAndTerritories'] == 'Cases_on_an_international_conveyance_Japan', ['countryterritoryCode']] = 'ICJ'
covid19.loc[covid19['countriesAndTerritories'] == 'Anguilla', ['countryterritoryCode']] = 'ANG'
covid19.loc[covid19['countriesAndTerritories'] == 'Bonaire, Saint Eustatius and Saba', ['countryterritoryCode']] = 'BON'
covid19.loc[covid19['countriesAndTerritories'] == 'Falkland_Islands_(Malvinas)', ['countryterritoryCode']] = 'FAL'
covid19.loc[covid19['countriesAndTerritories'] == 'Western_Sahara', ['countryterritoryCode']] = 'WSH'

### Betrachtung der Spalte 'popData2018'

In [ ]:
# Welche Instanzen weisen fehlende Werte auf?
mis_val_popData = covid19[covid19.popData2018.isnull()]
mis_val_popData

In [ ]:
# Wie viele Einträge je 'countriesAndTerritories' fehlen?
mis_val_popData['countriesAndTerritories'].value_counts()

--> die 5 betroffenen Ländern werden nun im Einzelfall betrachtet

#### Eritrea

In [ ]:
covid19[covid19.countriesAndTerritories == 'Eritrea'].head()

**Lösung**
- die Bevölkerungszahl wird manuell auf den Wert 6.500.000 gesetzt

Quelle:

"Im Jahr 2018 betrug die Einwohnerzahl von Eritrea geschätzt rund 6,05 Millionen Personen." (https://de.statista.com/statistik/daten/studie/417176/umfrage/gesamtbevoelkerung-von-eritrea/)

In [ ]:
covid19.loc[covid19['countriesAndTerritories'] == 'Eritrea', ['popData2018']] = 6500000

#### Anguilla

In [ ]:
covid19[covid19.countriesAndTerritories == 'Anguilla'].head()

**Lösung**
- die Bevölkerungszahl wird manuell auf den Wert 17.422 gesetzt

Quelle:

"Bevölkerung: 17.422 (Juli 2018 est.)" (https://www.indexmundi.com/de/anguilla/bevolkerung.html)

In [ ]:
covid19.loc[covid19['countriesAndTerritories'] == 'Anguilla', ['popData2018']] = 17422

#### Bonaire, Saint Eustatius and Saba

In [ ]:
covid19[covid19.countriesAndTerritories == 'Bonaire, Saint Eustatius and Saba'].head()

**Lösung**
- die Bevölkerungszahl wird manuell auf den Wert 25.157 gesetzt

Quelle:

"Einwohner Schätzung 2019-01-01: 25.157"
(https://www.citypopulation.de/de/caribbeannetherlands/)

In [ ]:
covid19.loc[covid19['countriesAndTerritories'] == 'Bonaire, Saint Eustatius and Saba', ['popData2018']] = 25157

#### Falkland Islands (Malvinas)

In [ ]:
covid19[covid19.countriesAndTerritories == 'Falkland_Islands_(Malvinas)'].head()

**Lösung**
- die Bevölkerungszahl wird manuell auf den Wert 2.922 gesetzt

Quelle:

"Einwohnerzahl: 2.922" (https://de.wikipedia.org/wiki/Falklandinseln)

In [ ]:
covid19.loc[covid19['countriesAndTerritories'] == 'Falkland_Islands_(Malvinas)', ['popData2018']] = 2922

#### Western_Sahara

In [ ]:
covid19[covid19.countriesAndTerritories == 'Western_Sahara'].head()

**Lösung**
- die Bevölkerungszahl wird manuell auf den Wert 554.200 gesetzt

Quelle:

"Einwohner Projektion (P) 2018-07-01: 554.200" (https://www.citypopulation.de/WesternSahara_d.html)

In [ ]:
covid19.loc[covid19['countriesAndTerritories'] == 'Western_Sahara', ['popData2018']] = 554200

## Überprüfen auf weitere Auffälligkeiten

### negative Erfassungszahlen für (Neu-)Infizierte und Todesfälle

#### Neuinfektionen

In [ ]:
covid19[covid19.cases < 0]

- mehrere Beobachtungen weisen negative Werte in der Spalte 'cases' auf
- diese erscheinen unplausibel und deuten auf eine Fehlerfassung hin

**Lösung**

- Umkehr der Vorzeichen

In [ ]:
# Funktion, die das Vorzeichen für negative Werte umkehrt
korr_negativ = lambda i: i*-1 if i < 0 else i

# Anwendung der Lambda-Funktion auf die Spalte 'cases'
covid19['cases'] = covid19['cases'].apply(korr_negativ)

#### Todesfälle

In [ ]:
covid19[covid19.deaths < 0]

-  Handlungsbedarf in einer Zeile

In [ ]:
# Anwendung der Lambda-Funktion auf die Spalte 'deaths'
covid19['deaths'] = covid19['deaths'].apply(korr_negativ)

# Visualisierung der Daten

## Die 10 am stärksten betroffenen Länder weltweit

Datenstand: 15.05.2020

### Gesamtinfektionen

In [ ]:
# im Dataframe covid19 werden die Einträge nach Ländern ('countriesAndTerritories') gruppiert
# das auf diese Weise neu entstandene Dataframe beinhaltet nun lediglich aggregierte Werte (Summen) und keine Zeitdimension mehr
# in einem Säulendiagramm werden die 10 Länder visualisiert, die die größte Anzahl an Gesamtinfektionen aufweisen
covid19.groupby('countriesAndTerritories')['cases'].sum().nlargest(10).plot(kind = 'bar', color = ['gray', 'gray', 'gray', 'gray', 'gray', 'gray', 'darkred', 'gray', 'gray', 'gray'])

# Einstellungen für Titel, Beschriftungen, Schriftgröße ...
plt.title('Fallzahlen der 10 am stärksten betroffenen Länder (weltweit)', fontsize = 24, pad = 30)
plt.ylabel('Gesamtinfektionen', fontsize = 20)
plt.xlabel('Land', fontsize = 20)
plt.xticks(fontsize = 18)
plt.yticks(fontsize = 18)

### Todesfälle

In [ ]:
# im Dataframe covid19 werden die Einträge nach Ländern ('countriesAndTerritories') gruppiert
# das auf diese Weise neu entstandene Dataframe beinhaltet nun lediglich aggregierte Werte (Summen) und keine Zeitdimension mehr
# in einem Säulendiagramm werden die 10 Länder visualisiert, die die größte Anzahl an Todesfällen aufweisen
covid19.groupby('countriesAndTerritories')['deaths'].sum().nlargest(10).plot(kind = 'bar', color = ['gray', 'gray', 'gray', 'gray', 'gray', 'gray', 'gray', 'darkred', 'gray', 'gray'])

# Einstellungen für Titel, Beschriftungen, Schriftgröße ...
plt.title('Fallzahlen der 10 am stärksten betroffenen Länder (weltweit)', fontsize = 24, pad = 30)
plt.ylabel('Todesfälle', fontsize = 20)
plt.xlabel('Land', fontsize = 20)
plt.xticks(fontsize = 18)
plt.yticks(fontsize = 18)

## Die 10 am stärksten betroffenen Länder Europas

Datenstand: 15.05.2020

### Gesamtinfektionen

In [ ]:
# Erstellung eines separaten Dataframes, das aus dem Dataframe covid19 alle Einträge für Europa herausfiltert
europa = covid19[covid19.continentExp == 'Europe']

In [ ]:
# im Dataframe europa werden die Einträge nach Ländern ('countriesAndTerritories') gruppiert
# das auf diese Weise neu entstandene Dataframe beinhaltet nun lediglich aggregierte Werte (Summen) und keine Zeitdimension mehr
# in einem Säulendiagramm werden die 10 Länder visualisiert, die die größte Anzahl an Gesamtinfektionen aufweisen
europa.groupby('countriesAndTerritories')['cases'].sum().nlargest(10).plot(kind = 'bar', color = ['gray', 'gray', 'gray', 'gray', 'darkred', 'gray', 'gray', 'gray', 'gray', 'gray'])

# Einstellungen für Titel, Beschriftungen, Schriftgröße ...
plt.title('Fallzahlen der 10 am stärksten betroffenen Länder (Europa)', fontsize = 24, pad = 30)
plt.ylabel('Gesamtinfektionen', fontsize = 20)
plt.xlabel('Land', fontsize = 20)
plt.xticks(fontsize = 18)
plt.yticks(fontsize = 18)

### Todesfälle

In [ ]:
# im Dataframe europa werden die Einträge nach Ländern ('countriesAndTerritories') gruppiert
# das auf diese Weise neu entstandene Dataframe beinhaltet nun lediglich aggregierte Werte (Summen) und keine Zeitdimension mehr
# in einem Säulendiagramm werden die 10 Länder visualisiert, die die größte Anzahl an Todesfällen aufweisen
europa.groupby('countriesAndTerritories')['deaths'].sum().nlargest(10).plot(kind = 'bar', color = ['gray', 'gray', 'gray', 'gray', 'gray', 'darkred', 'gray', 'gray', 'gray', 'gray'])

# Einstellungen für Titel, Beschriftungen, Schriftgröße ...
plt.title('Fallzahlen der 10 am stärksten betroffenen Länder (Europa)', fontsize = 24, pad = 30)
plt.ylabel('Todesfälle', fontsize = 20)
plt.xlabel('Land', fontsize = 20)
plt.xticks(fontsize = 18)
plt.yticks(fontsize = 18)

## Entwicklungen in Deutschland

Datenstand: 15.05.2020

In [2]:
# Erstellung eines separaten Dataframes, das aus dem Dataframe covid19 alle Einträge für Deutschland herausfiltert
# germany = covid19[covid19['countriesAndTerritories'] == 'Germany']
germany = pd.DataFrame(covid19[covid19['countriesAndTerritories'] == 'Germany'])

# Erstellung zweier zusätzlicher Spalten, die die kumulierten Gesamtinfektionen und Todesfälle aufzeigen
germany['Fälle kumuliert'] = germany['cases'].cumsum()
germany['Todesfälle kumuliert'] = germany['deaths'].cumsum()

NameError: name 'pd' is not defined

In [ ]:
# Kürzen des Dataframes, sodass die Einträge mit dem ersten registrierten Fall beginnen (Datum gesondert ermittelt)
# alle Einträge mit 0 vor dem 28.01.2020 wurden 'abgeschnitten'
germany = germany['2020-01-28' : ]

# Zeichnen von je einer Linie für Gesamtinfektionen und Todesfälle
plt.plot(germany['Fälle kumuliert'], label = 'Gesamtinfektionen', alpha = 0.7, color = 'black', lw = 3, marker = 'o')
plt.plot(germany['Todesfälle kumuliert'], label = 'Todesfälle', alpha = 0.7, color = 'darkred', lw = 3, marker = 'd')

# Einstellungen für Titel, Beschriftungen, Schriftgröße ...
plt.title('Pandemieentwicklung in Deutschland', fontsize = 24, pad = 30)
plt.ylabel('Anzahl', fontsize = 20)
plt.xticks(rotation = 45)
plt.xticks(fontsize = 18)
plt.yticks(fontsize = 18)
plt.legend(fontsize = 18)
plt.grid()

## Vergleich mit Großbritannien (höchste Anzahl Todesfälle in Europa), Russland (höchste europäische Infektionszahl) und China (Urspungsland)

Datenstand: 15.05.2020

In [ ]:
# Entfernen des Zeitindex zur Vergleichbarkeit der Entwicklungen mit anderen Ländern
germany = germany.reset_index()
del germany['dateRep']



# Erstellung eines separaten Dataframes, das aus dem Dataframe covid19 alle Einträge für China herausfiltert
china = pd.DataFrame(covid19[covid19['countriesAndTerritories'] == 'China'])
# Entfernen des Zeitindex zur Vergleichbarkeit der Entwicklungen mit anderen Ländern
china = china.reset_index()
del china['dateRep']
# Erstellung zweier zusätzlicher Spalten, die die kumulierten Gesamtinfektionen und Todesfälle aufzeigen
china['Fälle kumuliert'] = china['cases'].cumsum()
china['Todesfälle kumuliert'] = china['deaths'].cumsum()



# Erstellung eines separaten Dataframes, das aus dem Dataframe covid19 alle Einträge für Großbritannien herausfiltert
uk = pd.DataFrame(covid19[covid19['countriesAndTerritories'] == 'United_Kingdom'])
# Kürzen des Dataframes, sodass die Einträge mit dem ersten registrierten Fall beginnen (Datum gesondert ermittelt)
uk = uk['2020-01-31' :]
# Entfernen des Zeitindex zur Vergleichbarkeit der Entwicklungen mit anderen Ländern
uk = uk.reset_index()
del uk['dateRep']
# Erstellung zweier zusätzlicher Spalten, die die kumulierten Gesamtinfektionen und Todesfälle aufzeigen
uk['Fälle kumuliert'] = uk['cases'].cumsum()
uk['Todesfälle kumuliert'] = uk['deaths'].cumsum()



# Erstellung eines separaten Dataframes, das aus dem Dataframe covid19 alle Einträge für Russland herausfiltert
russia = pd.DataFrame(covid19[covid19['countriesAndTerritories'] == 'Russia'])
# Kürzen des Dataframes, sodass die Einträge mit dem ersten registrierten Fall beginnen (Datum gesondert ermittelt)
russia = russia['2020-02-01' :]
# Entfernen des Zeitindex zur Vergleichbarkeit der Entwicklungen mit anderen Ländern
russia = russia.reset_index()
del russia['dateRep']
# Erstellung zweier zusätzlicher Spalten, die die kumulierten Gesamtinfektionen und Todesfälle aufzeigen
russia['Fälle kumuliert'] = russia['cases'].cumsum()
russia['Todesfälle kumuliert'] = russia['deaths'].cumsum()

In [ ]:
# Zeichnen von je einer Linie für Gesamtinfektionen der 4 betrachteten Länder
plt.plot(russia['Fälle kumuliert'], label = 'Russland', alpha = 0.7, color = 'darkgreen', lw = 3, marker = 'o')
plt.plot(uk['Fälle kumuliert'], label = 'Großbritannien', alpha = 0.7, color = 'yellow', lw = 3, marker = 'o')
plt.plot(germany['Fälle kumuliert'], label = 'Deutschland', alpha = 0.7, color = 'darkblue', lw = 3, marker = 'o')
plt.plot(china['Fälle kumuliert'], label = 'China', alpha = 0.7, color = 'black', lw = 3, marker = 'o')

# Einstellungen für Titel, Beschriftungen, Schriftgröße ...
plt.xticks(fontsize = 18)
plt.yticks(fontsize = 18)
plt.legend(fontsize = 18)
plt.xlim(0,103)
plt.xticks(np.arange(0, 103, 7), range(16))
plt.xlabel('Woche', fontsize = 20)
plt.ylabel('Anzahl der Infizierten', fontsize = 20)
plt.title('Vergleich der Pandemieentwicklung in ausgewählten Ländern (Gesamtinfektionen)', fontsize = 24, pad = 30)
plt.grid()

In [ ]:
# Zeichnen von je einer Linie für Todesfälle der 4 betrachteten Länder
plt.plot(uk['Todesfälle kumuliert'], label = 'Großbritannien', alpha = 0.7, color = 'darkgreen', lw = 3, marker = 'd')
plt.plot(germany['Todesfälle kumuliert'], label = 'Deutschland', alpha = 0.7, color = 'darkblue', lw = 3, marker = 'd')
plt.plot(china['Todesfälle kumuliert'], label = 'China', alpha = 0.7, color = 'black', lw = 3, marker = 'd')
plt.plot(russia['Todesfälle kumuliert'], label = 'Russland', alpha = 0.7, color = 'yellow', lw = 3, marker = 'd')

# Einstellungen für Titel, Beschriftungen, Schriftgröße ...
plt.xticks(fontsize = 18)
plt.yticks(fontsize = 18)
plt.legend(fontsize = 18)
plt.xlim(0,103)
plt.xticks(np.arange(0, 103, 7), range(15))
plt.xlabel('Woche', fontsize = 20)
plt.ylabel('Anzahl der Verstorbenen', fontsize = 20)
plt.title('Vergleich der Pandemieentwicklung in ausgewählten Ländern (Todesfälle)', fontsize = 24, pad = 30)
plt.grid()

## Abhängigkeit zwischen Gesamtinfektionen und Todesfällen

Datenstand: 15.05.2020

In [ ]:
# in einem neuen Dataframe world werden aus covid19 die Einträge nach Ländern ('countriesAndTerritories') gruppiert und gespeichert
# alle numerischen Werte werden als Summe aggregiert
world = covid19.groupby(['countriesAndTerritories']).sum()

# Erstellung einer zusätzlicher Spalten für den Kontinent mit Initialwert '' und Setzen der Bevölkerungszahl auf 0
# durch die Aggregationsmethode .sum() werden auch die Bevölkerungszahlen addiert
# um die tatsächliche Bevölkerungszahl zu erhalten, wird sie im nächsten Schritt separat bestimmt
world['Kontinent'] = ''
world['popData2018'] = 0

# Anlegen einer Liste mit allen Ländern (jedes Land kommt nur einmal in dieser Liste vor)
land = list(covid19.countriesAndTerritories.unique())

# Bestimme für jedes Land in der Liste den Kontinent und die Bevölkerungszahl aus dem Dataframe covid19 und überschreibe die Einträge
# entsprechend im Dataframe world
for l in land:
  cont = covid19[covid19.countriesAndTerritories == l]['continentExp'].unique()
  cont = cont[0]
  world.loc[l, 'Kontinent'] = cont
  pop = covid19[covid19.countriesAndTerritories == l]['popData2018'].unique()
  pop = pop[0]
  world.loc[l, 'popData2018'] = pop

# Dictionary, das Kontinenten Farben zuordnet
col = {'Africa':'black',
       'America':'red',
       'Asia':'yellow',
       'Europe':'green',
       'Oceania':'blue',
       'Other' : 'purple'}

# Erstellung einer neuen Spalte, das jedem Land eine Farbe entsprechend des Kontinents zuordnet
world['Farbe'] = world.Kontinent.map(col)
# Erstellung weiterer Spalten für die Auswertung
#### Wie viel Prozent der Bevölkerung haben sich infiziert?
world['% Infizierte'] = world.cases / world.popData2018 * 100
#### Wie hoch ist die Mortalitätsrate bei den Infizierten?
world['% Verstorbene'] = world.deaths / world.cases * 100
#### Wie viele Fälle sind derzeit aktiv erkrankt oder geheilt?
world['Aktive Fälle inkl. Geheilte'] = world.cases - world.deaths

# Anzeigen des neuen Dataframes
world

In [ ]:
# Darstellung der Abhängigkeit zwischen Gesamtinfektionen und Todesfällen pro Land
# Farbe des Punktes ergibt repräsentiert den jeweiligen Kontinent
# Größe des Punktes ergibt sich aus der Bevölkerungszahl (um die Punkte in einer sinnvollen Größe darzustellen wurden sie mit dem Faktor
# 1/100000 herunterskaliert)
plt.scatter(world.deaths, world.cases, s = world.popData2018/100000, alpha = 0.4, c = world.Farbe)

# Beschriftungen einzelner Datenpunkte
liste = ['Germany', 'Russia', 'United_Kingdom', 'China', 'United_States_of_America']
for l in liste:
  x = world.loc[l]['deaths']
  y = world.loc[l]['cases']
  pylab.text(x , y, l, fontsize = 12)

# Benutzerdefinierte Legende
black_line = mlines.Line2D([], [], color='black',markersize=15, label='Afrika')
red_line = mlines.Line2D([], [], color='red', markersize=15, label='Amerika')
yellow_line = mlines.Line2D([], [], color='yellow', markersize=15, label='Asien')
green_line = mlines.Line2D([], [], color='green', markersize=15, label='Europa')
# blue_line = mlines.Line2D([], [], color='blue', markersize=15, label='Ozeanien')
# purple_line = mlines.Line2D([], [], color='purple', markersize=15, label='Other')
# handles = [black_line, red_line, yellow_line, green_line, blue_line, purple_line]
handles = [black_line, red_line, yellow_line, green_line]
labels = [h.get_label() for h in handles]
plt.legend(handles=handles, labels=labels, fontsize = 18)

# Einstellungen für Titel, Beschriftungen, Schriftgröße ...
plt.xlabel('Todesfälle', fontsize = 20)
plt.ylabel('Gesamtinfektionen', fontsize = 20)
plt.title('Abhängigkeit zwischen Gesamtinfektionen und Todesfällen der einzelnen Länder (mit USA)', fontsize = 24, pad = 30)
plt.xticks(fontsize = 18)
plt.yticks(fontsize = 18)

In [ ]:
# erneute Darstellung der Abhängigkeit zwischen Gesamtinfektionen und Todesfällen pro Land
# Farbe des Punktes ergibt repräsentiert den jeweiligen Kontinent
# Größe des Punktes ergibt sich aus der Bevölkerungszahl (um die Punkte in einer sinnvollen Größe darzustellen wurden sie mit dem Faktor
# 1/100000 herunterskaliert)
plt.scatter(world.deaths, world.cases, s = world.popData2018/100000, alpha = 0.4, c = world.Farbe)

# Setzen von Limits für die Achsenwerte, um in einen bestimmten Bereich zu zoomen
plt.xlim(0,35000)
plt.ylim(0,270000)

# Beschriftungen einzelner Datenpunkte
liste = ['Germany', 'Russia', 'United_Kingdom', 'China']
for l in liste:
  x = world.loc[l]['deaths']
  y = world.loc[l]['cases']
  pylab.text(x , y, l, fontsize = 12)

# Benutzerdefinierte Legende
black_line = mlines.Line2D([], [], color='black',markersize=15, label='Afrika')
red_line = mlines.Line2D([], [], color='red', markersize=15, label='Amerika')
yellow_line = mlines.Line2D([], [], color='yellow', markersize=15, label='Asien')
green_line = mlines.Line2D([], [], color='green', markersize=15, label='Europa')
# blue_line = mlines.Line2D([], [], color='blue', markersize=15, label='Ozeanien')
# purple_line = mlines.Line2D([], [], color='purple', markersize=15, label='Other')
# handles = [black_line, red_line, yellow_line, green_line, blue_line, purple_line]
handles = [black_line, red_line, yellow_line, green_line]
labels = [h.get_label() for h in handles]
plt.legend(handles=handles, labels=labels, fontsize = 18)

# Einstellungen für Titel, Beschriftungen, Schriftgröße ...
plt.xlabel('Todesfälle', fontsize = 20)
plt.ylabel('Gesamtinfektionen', fontsize = 20)
plt.title('Abhängigkeit zwischen Gesamtinfektionen und Todesfällen der einzelnen Länder (ohne USA)', fontsize = 24, pad = 30)
plt.xticks(fontsize = 18)
plt.yticks(fontsize = 18)

**ANMERKUNGEN**
- durch Veränderung der Werte für plt.xlim und plt.ylim kann der Plot beliebig angepasst und ein bestimmter Bereich hervorgehoben werden
- ggf. müssen die auskommentierten Zeilen für die Legendeneinträge wieder eingeblendet werden
- ggf. müssen die Datenbeschriftungen der Länder auskommentiert werden, wenn ein Bereich mit niedrigen Gesamtinfektionen und Todesfällen näher betrachtet werden soll
- überlappen die Datenpunkte nicht mehr so stark, kann es ebenfalls wünschenswert sein, sämtliche Datenbeschriftungen anzuzeigen

## Top 5 Mortalitätsrate

Datenstand: 15.05.2020

Fragestellung: In welchen Ländern sind vergleichsweise die meisten  Menschen verstorben (verglichen mit der Infiziertenzahl)?

- Schwellenwert: mindestens 50 registrierte Todesfälle im Land
- um Länder auszuschließen, deren Fall- und Verstorbenenzahlen noch nicht repräsentativ sind (z.B. Nicaragua, cases: 25, deaths: 8, Mortalitätsrate: 32.00%)

In [ ]:
# Erstellung eines separaten Dataframes, das aus dem Dataframe world die 5 größten Einträge für die Spalte "% Verstorbene" anzeigt
# unter vorheriger Anwendung eines Filters, der Länder mit weniger als 50 registrierten Todesfällen ausschließt
a = world[world.deaths >= 50]['% Verstorbene'].nlargest(5)

# Anzeigen von a
a

In [ ]:
import matplotlib as mpl
mpl.rcParams['font.size'] = 18

# Zeichne für jedes Land ein Kreisdiagramm mit dem Anteil der Verstorbenen innerhalb der Infizierten
for l in list(a.index):
  akt = world.loc[l]['Aktive Fälle inkl. Geheilte']
  tot = world.loc[l]['deaths']
  labels = 'Aktive Fälle inkl. Geheilte', 'Verstorbene'
  sizes = [akt, tot]
  colors = ['grey', 'darkred']

  plt.pie(sizes,
          labels=labels,
          colors=colors,
          autopct='%1.1f%%',
          shadow=False,
          startangle=90)
  plt.axis('equal')
  plt.title(l, fontsize = 20)
  plt.show()